# Import Libraries

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import os
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import files
from keras.preprocessing import image

# Download & unzip dataset

In [0]:
!wget -q  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip
!unzip -q rockpaperscissors.zip
!mv "/content/rockpaperscissors/rps-cv-images" "/content/"

base_dir = '/content/rockpaperscissors'

paper_dir = os.path.join(base_dir, 'paper')
rock_dir = os.path.join(base_dir, 'rock')
scissors_dir = os.path.join(base_dir, 'scissors')

# Train and validation data split

In [0]:
dataset = base_dir

data_generator = ImageDataGenerator(rescale=1/255, 
                                    validation_split=0.2)

training_generator = data_generator.flow_from_directory(
    dataset,
    target_size=(150, 150),
    batch_size=128,
    class_mode='categorical',
    subset='training'
)

validation_generator = data_generator.flow_from_directory(
    dataset,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

In [0]:
x, y = training_generator.next()

for i in range (10):
      plt.imshow(x[i], interpolation='nearest')
      plt.show()
      print(y[i])

# Create model using sequential model

In [0]:
model = tf.keras.models.Sequential([

    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.summary()

model.compile(optimizer='rmsprop',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train model

In [0]:
model_training = model.fit(training_generator, 
                    epochs=15, 
                    steps_per_epoch=12, 
                    validation_data = validation_generator, 
                    verbose = 1, 
                    validation_steps=3)

# Model training and validation accuracy graph

In [0]:
acc = model_training.history['accuracy']
val_acc = model_training.history['val_accuracy']
loss = model_training.history['loss']
val_loss = model_training.history['val_loss']
epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Train Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.title('Train and Validation Accuracy')
plt.legend(loc=0)
plt.figure()

# Image prediction

In [0]:
uploaded = files.upload()

for fn in uploaded.keys():
 
  path = fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images)
  class_names = ['paper', 'rock', 'scissors']

  print('Prediction result:' + ' ' + class_names[np.argmax(classes)])